In [71]:
import os
import string

import torch

1. read names.txt
2. add . at beginning and end
3. create bigrams (i, i+1) into a dictionary and update count
4. create a matrix (in torch) with first letter on rows and second letter on columns and create a count
5. convert the count to probability
6. create a name using probabilistci distribution

In [72]:

words = open('names.txt', 'r').read().splitlines()

with open("names.txt", 'r') as file:
    names = file.readlines()

bigram_count = {}

for n in names:
    n = "." + n.strip() + "."
    for i in range(len(n.strip()) - 1):
        bigram_string = "%s%s" % (n[i], n[i+1])
        bigram_count[bigram_string] = bigram_count.get(bigram_string, 0) + 1
    

In [73]:
itos = {}
stoi = {}

stoi['.'] = 0
itos[0] = '.'
for i,l in enumerate(sorted(list(set(string.ascii_letters.lower())))):
    itos[i+1] = l
    stoi[l] = i+1

In [74]:
C = torch.zeros((27,27), dtype=torch.int32)
for k,v in bigram_count.items():
    C[stoi[k[0]], stoi[k[1]]] = v

print(C)

tensor([[   0, 4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591, 2422, 2963,
         1572, 2538, 1146,  394,  515,   92, 1639, 2055, 1308,   78,  376,  307,
          134,  535,  929],
        [6640,  556,  541,  470, 1042,  692,  134,  168, 2332, 1650,  175,  568,
         2528, 1634, 5438,   63,   82,   60, 3264, 1118,  687,  381,  834,  161,
          182, 2050,  435],
        [ 114,  321,   38,    1,   65,  655,    0,    0,   41,  217,    1,    0,
          103,    0,    4,  105,    0,    0,  842,    8,    2,   45,    0,    0,
            0,   83,    0],
        [  97,  815,    0,   42,    1,  551,    0,    2,  664,  271,    3,  316,
          116,    0,    0,  380,    1,   11,   76,    5,   35,   35,    0,    0,
            3,  104,    4],
        [ 516, 1303,    1,    3,  149, 1283,    5,   25,  118,  674,    9,    3,
           60,   30,   31,  378,    0,    1,  424,   29,    4,   92,   17,   23,
            0,  317,    1],
        [3983,  679,  121,  153,  384, 1271,   82,

In [75]:
p = C[0].float()
p = p / p.sum()
p

tensor([0.0000, 0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273,
        0.0184, 0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029,
        0.0512, 0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])

In [76]:
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]

'j'

In [77]:
P = (C+1).float()
P /= P.sum(1, keepdims=True)
print(P[0])

tensor([3.1192e-05, 1.3759e-01, 4.0767e-02, 4.8129e-02, 5.2745e-02, 4.7785e-02,
        1.3038e-02, 2.0898e-02, 2.7293e-02, 1.8465e-02, 7.5577e-02, 9.2452e-02,
        4.9064e-02, 7.9195e-02, 3.5777e-02, 1.2321e-02, 1.6095e-02, 2.9008e-03,
        5.1154e-02, 6.4130e-02, 4.0830e-02, 2.4641e-03, 1.1759e-02, 9.6070e-03,
        4.2109e-03, 1.6719e-02, 2.9008e-02])


In [78]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out = []
    ix = 0
    while True:
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix==0:
            break
    print(''.join(out))

junide.
janasah.
p.
cony.
a.


In [79]:
log_likelihood = 0.0
n = 0

for w in words:
#for w in ["swapan"]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1, ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1
    #print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

log_likelihood=tensor(-559951.5625)
nll=tensor(559951.5625)
2.4543561935424805


----------------------------------------------------------------------------------------------------

## Using neural nets

1. First letter of bigram is input = x, second is output = y
2. create a NN with 27 neurons and 1 output, choose random weights
3. feed in x, get the y. Calculate the loss
4. back propgate loss and increase W with that gra
5. repeat 5 times to get proper weights
6. use the weights to predict the name

In [83]:
with open("names.txt", 'r') as file:
    names = file.readlines()

xs = []
ys = []

for n in names:
    n = "." + n.strip() + "."
    for i in range(len(n.strip()) - 1):
        xs.append(stoi[n[i]])
        ys.append(stoi[n[i+1]])

xs = torch.tensor(xs)
ys = torch.tensor(ys)

num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

number of examples:  228146


In [334]:
# gradeint descent

import torch.nn.functional as F


for i in range(1):

    #forward pass
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
    print(loss.item())
    

    #backward pass
    W.grad = None
    loss.backward()

    #update
    W.data += -50 * W.grad
    

2.482038736343384


In [338]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  
  out = []
  ix = 0
  while True:
    
    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------
    
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

junide.
janasah.
prelay.
a.
nn.
